Script for analysing output, including:  
* Transform the fitted data: generate a lattice parameter-temperature table for plotting  
* Visualise eigenvalue distribution: generate a matrix indicating the negative eigenvalues
* Visualise the temperature dependency of lattice parameters

By Spica. Vir., ICL, Mar 23 - 22. spica.h.zhou@gmail.com

In [ ]:
import numpy as np
from matplotlib import 

In [ ]:
class Exp_data:
    """
    Class expansion data, store the lattice parameters and their variation with temperature.
    """
    def read(self, inputfile):
        """
        Read the inputfile (QHA fit output) for data
        """
        
    

In [ ]:
class Hess_data:
    """
    Class Hessian data, store the eigenvalue and corresponding eigenvectors
    """
    def read(self, inputfile):
        """
        Read the input file (QHA fit output) for data
        """

In [ ]:
def readfile(inputfile):
    label_t = 'TEMPERATURE'
    label_fe = 'HELMHOLTZ'
    label_latt = 'IRREDUCIBLE'
    label_hess = 'EIGENVALUES'
    
    countline = 0
    
    while countline < len(inputfile):
        if label_t in inputfile[countline]:
    
    

In [ ]:
file = open('QHA_fit.dat', "r")
data_in = file.readlines()
file.close()

latt_exp, hess_eigs = readfile(data_in)